In [2]:
import pandas as pd

# データフレームの作成
data = {
    'Points limit': [500, 1000, 10000, 500, 1000, 10000],
    'Proposed (5%)': [1, 9, 0.083, 25, 33, 0.106],
    'ORB': [2, 10, 0.361, 26, 34, 0.576],
    # 他の列も同様に追加
}
df = pd.DataFrame(data)

# LaTeX形式で基本的なテーブルを出力
latex_table = df.to_latex(index=False)
print(latex_table)

\begin{tabular}{rrr}
\toprule
 Points limit &  Proposed (5\%) &    ORB \\
\midrule
          500 &          1.000 &  2.000 \\
         1000 &          9.000 & 10.000 \\
        10000 &          0.083 &  0.361 \\
          500 &         25.000 & 26.000 \\
         1000 &         33.000 & 34.000 \\
        10000 &          0.106 &  0.576 \\
\bottomrule
\end{tabular}



C:\Users\taab7\AppData\Local\Temp\ipykernel_20124\2096336193.py:13: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=False)


In [13]:
import csv
import os
import numpy as np
import matplotlib.pyplot as plt
import math

COMPARE_DESCRIPTORS = ["Proposed", "orb", "sphorb", "sift", "alike", ]
ABLATION_DESCRIPTORS = ["Proposed","Proposed01", "Proposed1", "Proposed10", "Ltspoint", "Ftspoint"] 
NOT_T_DESCRIPTORS = ["sphorb", "Ltspoint", "Proposed01", "Proposed03", "Proposed05", "Proposed1", "Proposed3", "Proposed", "Proposed10", "Proposed20", "Proposed_un", "Ftspoint"]
METHODS = ["", "t"]
PARAMS = ["R", "T"]
PARAMS_DICT = {"R": "Rotation", "T": "Translation"}
ALL_LOCS = ["Classroom", "Room", "Realistic", "Interior1", "Interior2", "Urban1", "Urban2", "Urban3", "Urban4"]
INDOORS = ["Classroom", "Room", "Realistic", "Interior1", "Interior2"]
OUTDOORS = ["Urban1", "Urban2", "Urban3", "Urban4"]
TIME_CATEGORYS = ["FP", "MC", "PE"]
DESCRIPTORS_DICT = {"Proposed1": "Proposed (1%)",
                    "Proposed01": "Proposed (0.1%)",
                    "Proposed03": "Proposed (0.3%)",
                    "Proposed05": "Proposed (0.5%)",
                    "Proposed3": "Proposed (3%)",
                    "Proposed": "Proposed (5%)",
                    "Proposed10": "Proposed (10%)",
                    "Proposed20": "Proposed (20%)",
                    "Proposed_un": "MNN (No Filter)",
                    "orb": "ORB", 
                    "sphorb": "SPHORB",
                    "sift": "SIFT",
                    "alike": "ALIKE",
                    "spoint": "SPoint",
                    "Ltspoint": "TSPoint+L",
                    "Ftspoint": "TSPoint+F"
                    }
METHODS_DICT = {"t": "T", "":""}

In [14]:
def read_csv_data(file_path):
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        data = [float(row[0]) for row in reader]
    return data

In [15]:
def make_latex_table_MAEMSE():
    for i, param in enumerate(PARAMS):
        for j, loc in enumerate(["indoor", "outdoor"]):
            if loc == "indoor":
                locs = INDOORS
            else:
                locs = OUTDOORS
            print(param, loc)
            maemse_data = {'Points limit': [500, 1000, 10000, 500, 1000, 10000]}
            maemse_compare_df = pd.DataFrame(maemse_data)
            maemse_ablation_df = pd.DataFrame(maemse_data)
            for tgt in ["compare", "ablation"]:
                if tgt == "compare":
                    descriptors = COMPARE_DESCRIPTORS
                    maemse_df = maemse_compare_df
                else:
                    descriptors = ABLATION_DESCRIPTORS
                    maemse_df = maemse_ablation_df
                for descriptor in descriptors:
                    for method in METHODS:
                        if descriptor in NOT_T_DESCRIPTORS and method == "t":
                            continue
                        method_tmp = f"{METHODS_DICT[method]}{DESCRIPTORS_DICT[descriptor]}"
                        maemse_df[method_tmp] = [0, 0, 0, 0, 0, 0]
        
            for idx_points, num_points in enumerate([500, 1000, 10000]):
                base_path = f"results/FP_{num_points}/values"
                value_dict = {}
                for tgt in ["compare", "ablation"]:
                    if tgt == "compare":
                        descriptors = COMPARE_DESCRIPTORS
                        maemse_df = maemse_compare_df
                    else:
                        descriptors = ABLATION_DESCRIPTORS
                        maemse_df = maemse_ablation_df
                    for descriptor in descriptors:
                        for method in METHODS:
                            if descriptor in NOT_T_DESCRIPTORS and method == "t":
                                continue
                            all_error_data = []
                            for scene in locs:
                                file_path = f"{base_path}/{scene}_{method}{descriptor}_5PA_GSM_wRT/{param}_ERRORS.csv"
                                error_data = read_csv_data(file_path)
                                all_error_data.extend(error_data)
                            method_tmp = f"{METHODS_DICT[method]}{DESCRIPTORS_DICT[descriptor]}"
                            maemse_df.loc[idx_points, method_tmp] = np.mean(all_error_data)
                            maemse_df.loc[idx_points+3, method_tmp] = np.mean(np.array(all_error_data) ** 2)
        
            maemse_latex_table = maemse_compare_df.to_latex(index=False,  float_format="{:.3f}".format)
            print(maemse_latex_table)
            maemse_latex_table = maemse_ablation_df.to_latex(index=False, float_format="{:.3f}".format)
            print(maemse_latex_table)

make_latex_table_MAEMSE()

R indoor
\begin{tabular}{rrrrrrrrr}
\toprule
 Points limit &  Proposed (5\%) &   ORB &  TORB &  SPHORB &  SIFT &  TSIFT &  ALIKE &  TALIKE \\
\midrule
          500 &          0.083 & 0.914 & 0.667 &   0.786 & 0.544 &  0.411 &  2.975 &   2.782 \\
         1000 &          0.092 & 0.832 & 0.559 &   0.526 & 0.281 &  0.229 &  2.923 &   2.681 \\
        10000 &          0.083 & 0.361 & 0.184 &   0.246 & 0.129 &  0.125 &  2.847 &   2.677 \\
          500 &          0.078 & 1.860 & 1.218 &   1.685 & 0.932 &  0.868 &  9.305 &   8.650 \\
         1000 &          0.123 & 1.811 & 1.144 &   1.082 & 0.378 &  0.495 &  9.129 &   8.300 \\
        10000 &          0.106 & 0.576 & 0.276 &   0.460 & 0.105 &  0.214 &  8.872 &   8.286 \\
\bottomrule
\end{tabular}

\begin{tabular}{rrrrrrr}
\toprule
 Points limit &  Proposed (5\%) &  Proposed (0.1\%) &  Proposed (1\%) &  Proposed (10\%) &  TSPoint+L &  TSPoint+F \\
\midrule
          500 &          0.083 &            0.107 &          0.088 &           1.091 

C:\Users\taab7\AppData\Local\Temp\ipykernel_20124\3809608482.py:49: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  maemse_latex_table = maemse_compare_df.to_latex(index=False,  float_format="{:.3f}".format)
C:\Users\taab7\AppData\Local\Temp\ipykernel_20124\3809608482.py:51: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  maemse_latex_table = maemse_ablation_df.to_latex(index=False, float_format="{:.3f}".format)
C:\Users\taab7\AppData\Local\Temp\ipykernel_20124\3809608482.py:49

In [13]:
def make_latex_table_time():
    for num_points in [500, 1000, 10000]:
        base_path = f"results/FP_{num_points}/values"
        value_dict = {}
        for tgt in ["compare", "ablation"]:
            if tgt == "compare":
                descriptors = COMPARE_DESCRIPTORS
            else:
                descriptors = ABLATION_DESCRIPTORS


            for time_category in TIME_CATEGORYS:
                print(time_category)
                for descriptor in descriptors:
                    for method in METHODS:
                        if descriptor in NOT_T_DESCRIPTORS and method == "t":
                            continue
                        all_time_data = []
                        for scene in ALL_LOCS:
                            file_path = f"{base_path}/{scene}_{method}{descriptor}_5PA_GSM_wRT/TIMES_{time_category}.csv"
                            time_data = read_csv_data(file_path)
                            all_time_data.extend(time_data)
                        print(method, descriptor, np.mean(all_time_data))
    

make_latex_table_time()

FP
 Proposed 0.5277161926269531
 orb 0.23602019938138816
t orb 0.23618378652594027
 sphorb 0.17404283057802764
 sift 0.2494446925701801
t sift 0.6942731138664298
 spoint 0.24164986705780028
t spoint 0.49267350912094116
 alike 0.20441039936076394
t alike 0.33773551120758055
MC
 Proposed 0.010328376197814942
 orb 0.010580351528448936
t orb 0.012928668791667009
 sphorb 0.0007817210295261481
 sift 0.009923319045885202
t sift 0.010582573648501428
 spoint 0.011118914604187012
t spoint 0.011753302192687989
 alike 0.010223396328834585
t alike 0.010941674995422363
PE
 Proposed 0.11742448272705078
 orb 0.14029738450279602
t orb 0.12187404863727398
 sphorb 0.1244141626625489
 sift 0.03531144031446242
t sift 0.037565813512636034
 spoint 0.015833193016052245
t spoint 0.013866042709350586
 alike 0.11945559862673594
t alike 0.11274332790374755
FP
 Proposed 0.5277161926269531
 Proposed01 0.5245466610908508
 Proposed1 0.5251558550834656
 Proposed10 0.4989517106056213
 Ltspoint 0.4897983779907227
 Ftspo